#Gold Layer Creation

## Read Files From Silver Layer

In [0]:
service_credential = dbutils.secrets.get(scope="databricks-app-kv",key="databricks-application")

spark.conf.set("fs.azure.account.auth.type.20230821desa.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.20230821desa.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.20230821desa.dfs.core.windows.net", "de4ff859-02b1-4e2f-9d16-b578fa03df4f")
spark.conf.set("fs.azure.account.oauth2.client.secret.20230821desa.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.20230821desa.dfs.core.windows.net", "https://login.microsoftonline.com/33da9f3f-4c1a-4640-8ce1-3f63024aea1d/oauth2/token")

###Load Actor/Repos/Orgs Table

In [0]:
actor_table = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/actors/*.parquet")
repos_table = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/repos/*.parquet")
orgs_table = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/orgs/*.parquet")

###Load Event Tables

In [0]:
event_tables = {}

event_tables["CreateEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/CreateEvent/*.parquet")

event_tables["CommitCommentEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/CommitCommentEvent/*.parquet")

event_tables["DeleteEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/DeleteEvent/*.parquet")

event_tables["ForkEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/ForkEvent/*.parquet")

event_tables["GollumEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/GollumEvent/*.parquet")

event_tables["IssueCommentEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/IssueCommentEvent/*.parquet")

event_tables["IssuesEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/IssuesEvent/*.parquet")

event_tables["MemberEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/MemberEvent/*.parquet")

event_tables["PublicEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/PublicEvent/*.parquet")

event_tables["PushEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/PushEvent/*.parquet")

event_tables["PullRequestEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/PullRequestEvent/*.parquet")

event_tables["PullRequestReviewCommentEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/PullRequestReviewCommentEvent/*.parquet")

event_tables["PullRequestReviewEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/PullRequestReviewEvent/*.parquet")

event_tables["ReleaseEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/ReleaseEvent/*.parquet")

event_tables["WatchEvent"] = spark.read.parquet(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/SilverLayer/WatchEvent/*.parquet")

##Create Gold Tables

In [0]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import col
from pyspark.sql.types import StructType

In [0]:
sub_tables = {}
fact_tables = {}

####Rename columns in actor, repo, and org tables

In [0]:
orgs_table = orgs_table.withColumnsRenamed({'org_id': 'id', 'org_login': 'login', 'org_url' :'url', 'org_avatar_url': 'avatar_url'})
actor_table = actor_table.withColumnsRenamed({'actor_id': 'id', 'actor_login': 'login', 'actor_url': 'url', 'actor_avatar': 'avatar', 'actor_display_login': 'display_login'})
repos_table = repos_table.withColumnsRenamed({'repo_id': 'id', 'repo_name': 'name', 'repo_url': 'url'})

###Create PushEvent Tables

In [0]:
PushEvent_df = event_tables['PushEvent']
PushEvent_fact = PushEvent_df.select('id', 'created_at', 'public', 'type', 'actor_id', 'repo_id', "org_id", "push_id")
PushEvent_payload = PushEvent_df.select('push_id', 'before', 'commits', 'distinct_size', 'head', 'ref', 'size')

fact_tables['PushEvent_fact'] = PushEvent_fact
sub_tables['PushEvent_payload'] = PushEvent_payload

### Create GollumEvent Tables

In [0]:

GollumEvent_df = event_tables['GollumEvent']
GollumEvent_fact = GollumEvent_df.select("id","created_at","public","type","actor_id","repo_id","org_id")
pages = GollumEvent_df.select('id', explode('pages').alias("page"))
pages = pages.select('id', 'page.*')
pages = pages.withColumnRenamed('id', 'event_id')

fact_tables['GollumEvent_fact'] = GollumEvent_fact
sub_tables['pages'] = pages

###Create ReleaseEvent Tables

In [0]:
ReleaseEvent_df = event_tables['ReleaseEvent']
ReleaseEvent_fact = ReleaseEvent_df.select('id', 'created_at', 'public', 'type', 'actor_id', 'org_id', 'repo_id', col('release.id').alias('release_id'))
releases = ReleaseEvent_df.select(col("release.*"), "action")
releases = releases.drop("assets_url", "discussion_url",'html_url','is_short_description_html_truncated','reactions','short_description_html','tarball_url','upload_url','url','zipball_url')
authors = releases.select(col("author.*"))
authors = authors.drop("avatar_url", "events_url", "followers_url", "following_url", "gists_url", "gravatar_id", "html_url", "starred_url", "subscriptions_url", "received_events_url", "repos_url")
releases = releases.withColumn("author", col("author.id"))

fact_tables['ReleaseEvent_fact'] = ReleaseEvent_fact
sub_tables['members'] = authors #author.id refrences members table
sub_tables['releases'] = releases 

###Create CommitCommentEvent Tables

In [0]:
CommitCommentEvent_df = event_tables['CommitCommentEvent']
CommitCommentEvent_fact = CommitCommentEvent_df.select('id', 'created_at', 'public', 'type', 'actor_id', 'repo_id', 'org_id', col("comment.id").alias("comment_id"))
comments = CommitCommentEvent_df.select("comment.*")
comments = comments.drop('_links','html_url','issue_url','performed_via_github_app','pull_request_url','reactions','url')

fact_tables['CommitCommentEvent_fact'] = CommitCommentEvent_fact
sub_tables['comments'] = comments

###Create PullRequestReviewEvent Tables

In [0]:
PullRequestReviewEvent_df = event_tables['PullRequestReviewEvent']
PullRequestReviewEvent_fact = PullRequestReviewEvent_df.select("id", "created_at", "public", "type", "actor_id", "org_id", "repo_id", col("pull_request.id").alias("pull_request_id"), col("review.id").alias("review_id"))

pull_requests = PullRequestReviewEvent_df.select("pull_request.*")
reviews = PullRequestReviewEvent_df.select("review.*")
pull_requests = pull_requests.drop('_links','assignee','assignees','auto_merge','base','comments_url','commits_url','diff_url','head','html_url','merge_commit_sha','mergable','mergeable_state','merged','merged_at','merged_by','milestone','patch_url','requested_reviewers','requested_teams','review_comment_url','review_comments_url','url')
reviews = reviews.drop('_links','html_url','pull_request_url')

fact_tables['PullRequestReviewEvent_fact'] = PullRequestReviewEvent_fact
sub_tables['pull_requests'] = pull_requests
sub_tables['reviews'] = reviews

### Create CreateEvent Tables

In [0]:
CreateEvent_df = event_tables['CreateEvent']
CreateEvent_fact = CreateEvent_df.select("id","public",'type','actor_id','repo_id','org_id','created_at', col('id').alias("create_event_payload_id"))
CreateEvent_payload = CreateEvent_df.select('id', 'master_branch','pusher_type','ref_type', 'ref')

fact_tables['CreateEvent_fact'] = CreateEvent_fact
sub_tables['CreateEvent_payload'] = CreateEvent_payload

### Create PullRequestReviewCommentEvent Tables

In [0]:
PullRequestReviewCommentEvent_df = event_tables['PullRequestReviewCommentEvent']
PullRequestReviewCommentEvent_fact = PullRequestReviewCommentEvent_df.select("id", "created_at", "public", "type", "actor_id", "repo_id", "org_id", col("id").alias("pull_request_review_comment_payload_id"))
PullRequestReviewCommentEvent_payload = PullRequestReviewCommentEvent_df.select("id", col("comment.id").alias("comment_id"), col("pull_request.id").alias("pull_request_id"))
comments = PullRequestReviewCommentEvent_df.select("comment.*")
pull_requests = PullRequestReviewCommentEvent_df.select("pull_request.*")
comments = comments.drop('_links','html_url','issue_url','performed_via_github_app','pull_request_url','reactions','url')
pull_requests = pull_requests.drop('_links','assignee','assignees','auto_merge','base','comments_url','commits_url','diff_url','head','html_url','merge_commit_sha','mergable','mergeable_state','merged','merged_at','merged_by','milestone','patch_url','requested_reviewers','requested_teams','review_comment_url','review_comments_url','url')

fact_tables['PullRequestReviewCommentEvent_fact'] = PullRequestReviewCommentEvent_fact
sub_tables['PullRequestReviewCommentEvent_payload'] = PullRequestReviewCommentEvent_payload
sub_tables['comments'] = sub_tables['comments'].union(comments)
sub_tables['pull_requests'] = sub_tables['pull_requests'].union(pull_requests)


###Create PublicEvent Tables

In [0]:
PublicEvent_df = event_tables['PublicEvent']
PublicEvent_fact = PublicEvent_df

fact_tables['PublicEvent_fact'] = PublicEvent_fact

###Create MemberEvent Tables

In [0]:
MemberEvent_df = event_tables['MemberEvent']
MemberEvent_fact = MemberEvent_df.select("id","created_at","public","type","actor_id","org_id","repo_id","action", col("member.id").alias("member_id"))
members = MemberEvent_df.select("member.*")
members = members.drop("avatar_url", "events_url", "followers_url", "following_url", "gists_url", "gravatar_id", "html_url", "starred_url", "subscriptions_url", "received_events_url", "repos_url")

fact_tables['MemberEvent_fact'] = MemberEvent_fact
sub_tables['members'] = sub_tables['members'].union(members)

###Create WatchEvent Tables

In [0]:
WatchEvent_df = event_tables['WatchEvent']
WatchEvent_fact = WatchEvent_df

fact_tables['WatchEvent_fact'] = WatchEvent_fact

###Create PullRequestEvent Tables

In [0]:
PullRequestEvent_df = event_tables['PullRequestEvent']
PullRequestEvent_fact = PullRequestEvent_df.select("id","created_at","public","type","actor_id", "org_id", "repo_id","action", col("pull_request.id").alias("pull_request_id"))
pull_requests =  PullRequestEvent_df.select("pull_request.*")
pull_requests = pull_requests.drop('_links','assignee','assignees','auto_merge','base','comments_url','commits_url','diff_url','head','html_url','merge_commit_sha','mergable','mergeable_state','merged','merged_at','merged_by','milestone','patch_url','requested_reviewers','requested_teams','review_comment_url','review_comments_url','url')

fact_tables['PullRequestEvent_fact'] = PullRequestEvent_fact
sub_tables['pull_requests'] = sub_tables['pull_requests'].union(pull_requests)

###Create IssueCommentEvent Tables

In [0]:
IssueCommentEvent_df = event_tables["IssueCommentEvent"]
IssueCommentEvent_fact = IssueCommentEvent_df.select('id','created_at','public','type','actor_id','repo_id','org_id', col("comment.id").alias('comment_id'), col('issue.id').alias('issue_id'))
comments = IssueCommentEvent_df.select("comment.*")
issues = IssueCommentEvent_df.select("issue.*")
comments = comments.drop('_links','html_url','issue_url','performed_via_github_app','pull_request_url','reactions','url')
issues = issues.drop('assignee','assignees','body','comments_url','events_url','html_url','labels','labels_url','locked','milestone','performed_via_github_app','reactions','repository_url','timeline_url','url','pull_request')

fact_tables['IssueCommentEvent_fact'] = IssueCommentEvent_fact
sub_tables['issues'] = issues
sub_tables['comments'] = sub_tables['comments'].union(comments)

###Create ForkEvent Tables

In [0]:
ForkEvent_df = event_tables['ForkEvent']
ForkEvent_fact = ForkEvent_df.select('id','created_at','public','type','actor_id','org_id','repo_id',col('forkee.id').alias('forkee_id'))
ForkEvent_payload = ForkEvent_df.select('forkee.*')
ForkEvent_payload = ForkEvent_payload.drop("license", "owner",'archive_url','assignees_url','blobs_url','branches_url','clone_url','collaborators')

fact_tables['ForkEvent_fact'] = ForkEvent_fact
sub_tables['ForkEvent_payload'] = ForkEvent_payload

###Create DeleteEvent Tables


In [0]:
DeleteEvent_df = event_tables['DeleteEvent']
DeleteEvent_fact = DeleteEvent_df.select('id', 'type', 'public', 'created_at', 'actor_id', 'org_id', 'repo_id')

fact_tables['DeleteEvent_fact'] = DeleteEvent_fact

###Create IssuesEvent Tables

In [0]:
IssuesEvent_df = event_tables['IssuesEvent']
IssuesEvent_fact = IssuesEvent_df.select('id','created_at','public','type','actor_id','org_id','repo_id','action',col('issue.id').alias('issue_id'))
issues = IssuesEvent_df.select('issue.*')
issues = issues.drop('assignee','assignees','body','comments_url','events_url','html_url','labels','labels_url','locked','milestone','performed_via_github_app','reactions','repository_url','timeline_url','url','pull_request')

fact_tables['IssueEvent_fact'] = IssuesEvent_fact
sub_tables['issues'] = sub_tables['issues'].union(issues)

##Union Fact Tables

In [0]:
fact_table = spark.createDataFrame([], StructType([]))
for name, table in fact_tables.items():
    fact_table = fact_table.unionByName(fact_tables[name], allowMissingColumns=True)

##Write Gold Layer Tables

###Save Fact Table

In [0]:
fact_table.write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/fact_table")

###Save Actor/Orgs/Repos Tables

In [0]:
actor_table.repartition(1).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/actors")
orgs_table.repartition(1).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/orgs")
repos_table.repartition(1).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/repos")

###Save Sub-Tables

In [0]:
sub_tables['PushEvent_payload'].repartition(120).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/PushEvent_payload")

sub_tables['pages'].repartition(1).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/pages")

sub_tables['members'].repartition(1).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/members")

sub_tables['releases'].repartition(1).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/releases")

sub_tables['comments'].repartition(30).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/comments")

sub_tables['pull_requests'].repartition(100).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/pull_requests")

sub_tables['reviews'].repartition(5).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/reviews")

sub_tables['CreateEvent_payload'].repartition(1).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/CreateEvent_payload")

sub_tables['PullRequestReviewCommentEvent_payload'].repartition(1).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/PullRequestReviewCommentEvent_payload")

sub_tables['issues'].repartition(13).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/issues")

sub_tables['ForkEvent_payload'].repartition(15).write.format("parquet").mode("overwrite").save(f"abfss://team2-project2@20230821desa.dfs.core.windows.net/GoldLayer/ForkEvent_payload")


1
2
3
4
5
6
7
8
9
10
